# Document Reader - Implantação

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
from aux_functions import build_result_dataframe

class Model:
    
    def __init__(self):
        self.loaded = False
        
    def load(self):
        artifacts = joblib.load("/tmp/data/document_reader.joblib")
        self.model = artifacts["model"]
        self.columns = artifacts["columns"]
        self.question = artifacts["question"]
        self.ntops_overall = artifacts["ntops_overall"]
        self.reader_score_weight = artifacts["reader_score_weight"]
        self.retriever_reader_pipeline = artifacts["retriever_reader_pipeline"]
        self.remove_no_answer_found = artifacts["remove_no_answer_found"]
        self.column_doc_id = artifacts["column_doc_id"]
        self.column_context = artifacts["column_context"]
        self.column_retriever_score = artifacts["column_retriever_score"]
        self.column_reader_score = artifacts["column_reader_score"]
        self.loaded = True

    def predict(self, X, feature_names, meta=None):
        
        if not self.loaded:
            self.load()
            
        if not feature_names:
            feature_names = self.columns
            
        
        df_input = pd.DataFrame(X,columns=feature_names)
        contexts_dict = {row[self.column_doc_id]:row[self.column_context] for i,row in df_input.iterrows()}
        df_result = self.model.forward(question = question,
                                          topn_contexts = list(contexts_dict.values()),
                                          top_ids = list(contexts_dict.keys()))

        df_result = build_result_dataframe(df_input = df_input,
                                           df_result = df_result,
                                           ntops_overall = self.ntops_overall,
                                           reader_score_weight = self.reader_score_weight,
                                           retriever_reader_pipeline = self.retriever_reader_pipeline,
                                           remove_no_answer_found = self.remove_no_answer_found,
                                           column_doc_id = self.column_doc_id,
                                           column_retriever_score = self.column_retriever_score,
                                           column_reader_score = self.column_reader_score)
        
        
        
        return df_result.to_numpy()

Overwriting Model.py


In [2]:
# import pandas as pd
# df = pd.read_csv("/tmp/data/fabc-step4-4.csv")
# X = df.to_numpy()

In [3]:
# from Model import Model
# model = Model()
# result = model.predict(X,None)
# result

array([[0,
        'HB19PR_SO06_SO07_SO08_SP_SO09 - Eficacia do herbicida imazethapyr associado a um inibidor da PROTOX para manejo de plantas daninhas na pre-emergencia da soja tolerante a glyphosate.pdf',
        '_raiz',
        'Setor de Herbologia HERBICIDAS RESIDUAIS NA CULTURA DA SOJA Eficácia do herbicida imazethapyr associado a um inibidor da PROTOX para manejo de plantas daninhas na pré-emergência da soja tolerante a glyphosate Autor: Luís Henrique Penckowski Eliana Fernandes Borsato Evandro H. G. Maschietto AVISO LEGAL Este documento está protegido por direitos autorais e pode conter informações confidenciais e/ou privilegiadas. É expressamente proibido copiar, modificar, distribuir, remover, adicionar ou divulgar o seu conteúdo, ou parte deste, em qualquer meio, sem o consentimento expresso e por escrito da FUNDAÇÃO ABC. Qualquer utilização das informações/dados de forma diversa do contido no presente documento afeta a precisão dos resultados e não reflete as conclusões da 